In [ ]:
#IMPORTING LIBRARIES
import pandas as pd
import numpy as np
from datetime import datetime, date
import matplotlib.pyplot as plt
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from scipy.special import softmax

In [ ]:
df = pd.read_csv("all_companies_tweets.csv")

In [ ]:
df.shape

(3326194, 13)

FILTERING 6 STOCKS BY THE NUMBER OF LIKES AND RETWEETS

In [ ]:
apple_filter = (df['apple'] == True)
condition = (df['retweet_num'] >= 30) | (df['like_num'] >= 50)
apple_reduced = df.loc[apple_filter & condition].reset_index()
apple_reduced.shape

(4080, 14)

In [ ]:
amazon_filter = (df['amazon'] == True)
condition = (df['retweet_num'] >= 30) | (df['like_num'] >= 50)
amazon_reduced = df.loc[amazon_filter & condition].reset_index()
amazon_reduced.shape

(2890, 14)

In [ ]:
tesla_filter = (df['tesla'] == True)
condition = (df['retweet_num'] >= 100) | (df['like_num'] >= 200)
tesla_reduced = df.loc[tesla_filter & condition].reset_index()
tesla_reduced.shape

(2904, 14)

In [ ]:
microsoft_filter = (df['microsoft'] == True)
condition = (df['retweet_num'] >= 30) | (df['like_num'] >= 50)
microsoft_reduced = df.loc[microsoft_filter & condition].reset_index()
microsoft_reduced.shape

(1000, 14)

In [ ]:
google_filter = (df['google'] == True)
condition = (df['retweet_num'] >= 30) | (df['like_num'] >= 50)
google_reduced = df.loc[google_filter & condition].reset_index()
google_reduced.shape

(1481, 14)

In [ ]:
netflix = np.zeros(df.shape[0],dtype=bool)
for i in range(df.shape[0]):
  if "netflix" in df.text[i].lower():
    netflix[i] = True
df["netflix"] = netflix
netflix_filter = (df['netflix'] == True)
condition = (df['retweet_num'] >= 5) | (df['like_num'] >= 5)
netflix_reduced = df.loc[netflix_filter & condition]
netflix_reduced.shape

(942, 14)

LOADING ROBERTA MODEL


In [ ]:
# Loading the pre-trained model and tokenizer
MODEL = "cardiffnlp/twitter-roberta-base-sentiment-latest"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
model = AutoModelForSequenceClassification.from_pretrained(MODEL)

# Function to classify tweets in batches
def classify_tweets_in_batches(tweet_list, batch_size=10):
    num_tweets = len(tweet_list)
    for i in range(0, num_tweets, batch_size):
        batch_tweets = list(tweet_list[i:i+batch_size])
        encoded_inputs = tokenizer(batch_tweets, return_tensors='pt', padding=True, truncation=True)
        outputs = model(**encoded_inputs)
        scores = outputs.logits
        probabilities = softmax(scores.detach().numpy(), axis=1)
        predicted_labels = np.argmax(probabilities, axis=1)
        sentiment_labels = [model.config.id2label[label_id] for label_id in predicted_labels]
        for label in sentiment_labels:
            yield label #yiled is used instead of return due to RAM memory considerations

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/929 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/501M [00:00<?, ?B/s]

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


APPLYING ROBERTA MODEL AND SAVING THE SENTIMENT LABELED DATAFRAME SEPARATELY FOR EACH STOCK


ROBERTA FOR NETFLIX

In [ ]:
tweets = netflix_reduced.text.values
sentiment_labels = list(classify_tweets_in_batches(tweets))
netflix_reduced["sentiment"] = sentiment_labels
netflix_reduced.to_csv("netflix_reduced_sentiment.csv",index=False)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
<ipython-input-18-117ce1f06f4e>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  netflix_reduced["sentiment"] = sentiment_labels


ROBERTA FOR AMAZON

In [ ]:
tweets = amazon_reduced.text.values
sentiment_labels = list(classify_tweets_in_batches(tweets))
amazon_reduced["sentiment"] = sentiment_labels
amazon_reduced.to_csv("amazon_reduced_sentiment.csv",index=False)

ROBERTA FOR APPLE

In [ ]:
tweets = apple_reduced.text.values
sentiment_labels = list(classify_tweets_in_batches(tweets))
apple_reduced["sentiment"] = sentiment_labels
apple_reduced.to_csv("apple_reduced_sentiment.csv",index=False)

ROBERTA FOR MICROSOFT

In [ ]:
tweets = microsoft_reduced.text.values
sentiment_labels = list(classify_tweets_in_batches(tweets))
microsoft_reduced["sentiment"] = sentiment_labels
microsoft_reduced.to_csv("microsoft_reduced_sentiment.csv",index=False)

ROBERTA FOR GOOGLE

In [ ]:
tweets = google_reduced.text.values
sentiment_labels = list(classify_tweets_in_batches(tweets))
google_reduced["sentiment"] = sentiment_labels
google_reduced.to_csv("google_reduced_sentiment.csv",index=False)

ROBERTA FOR TESLA

In [ ]:
tweets = tesla_reduced.text.values
sentiment_labels = list(classify_tweets_in_batches(tweets))
tesla_reduced["sentiment"] = sentiment_labels
tesla_reduced.to_csv("tesla_reduced_sentiment.csv",index=False)